In [1]:
# environment
import os
import sys
root_path = os.path.abspath("../")
sys.path.append(root_path)

In [2]:
import pandas as pd

In [3]:
from etls.sheets.pat_pop import PatpopScrub
from etls.sheets.va_cols import VARuleReader, get_col_map_all, get_all_rules
from etls.etl_helpers import try_strip, try_int_to_str
from etls.instructions import instructions

In [4]:
def display_check(result):
    old_col = result[0]
    new_cols = result[1]
    diff = old_col.compare(
        new_cols[0], result_names=(f"{old_col.name}", new_cols[0].name)
    )
    if len(new_cols) > 1:
        idx = list(diff.index)
        cols = [old_col] + new_cols
        return pd.DataFrame(cols).T.loc[idx]
    return diff

In [1]:
3.str.contains("hi")

SyntaxError: invalid decimal literal (3828878315.py, line 1)

In [5]:
df = pd.read_csv("../data/interim/ophthafterdxdate.csv")

C:\Users\PANDA15-User\AppData\Local\Temp\ipykernel_35024\1523278632.py:1: DtypeWarning: Columns (6,10,11,28,32,40,42,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/interim/ophthafterdxdate.csv")


## Going through each Visual Accuity column one by one and checking the edits. 
- "n/a" is converted to NaN when we convert .xlsx to .csv (already handled; subtract from expected)
- leading and trailing spaces are removed when we convert .xlsx to .csv (the diff between actual and expected should be added)


In [6]:
col_name = "VA_Right_Distance_SC"

In [7]:
col = df[col_name]
ins = instructions["ophthafterdxdate"][col_name]

In [8]:
result = PatpopScrub(col, ins).clean()

In [9]:
old = result[0]

In [10]:
new=result[1][0]

In [11]:
old_new = pd.DataFrame([old, new]).T
old_new.columns = ["old", "new"]

In [22]:
old_new[~old_new["old"].isna() & old_new["old"].str.contains("no_lookup")]

,old,new


In [13]:
on_str = old_new[~old_new["old"].isna() 
        & (old_new["old"].apply(type) == str)
        ]

In [14]:
on_str[on_str["old"].apply(len) <=3]["old"].value_counts()

old
HM     226
LP      82
NLP     57
CF      40
NT      12
cf       6
hm       4
UTO      3
72       2
X        2
nlp      1
200      1
Name: count, dtype: int64

In [13]:
old_new[~old_new["old_VA_Right_Distance_SC"].isnull()]

,old_VA_Right_Distance_SC,VA_Right_Distance_SC
20,20/70,20/70
21,20/70,20/70
22,20/50-,20/50
77,New glasses Rx,NaN
120,20/60,20/60
...,...,...
108360,20/60,20/60
108361,20/60,20/60
108362,20/50,20/50
108363,20/70,20/70


In [15]:
check = display_check(result)

In [16]:
len(check)

2537

In [39]:
try_int_to_str(9)

'9'

In [41]:
get_col_map_all(ins["args"])["CF @ 2 feet"]

'Count fingers'

In [47]:
def try_strip(x):
    """strip white space around strings, and convert solely whitespace to np.nan"""
    try:
        res = x.strip()
    except:
        res = x
    return np.nan if res == "" else res

In [48]:
try_strip(check[~check["VA_Right_Distance_SC"].isna() & check["VA_Right_Distance_SC"].str.contains("no_lookup")]["old_VA_Right_Distance_SC"][1402])

'CF @ 2 feet'

In [10]:
len(check) == 2541

False

### VA Left Distance_SC

In [18]:
col_name = "VA_Left_Distance_SC"
col = df[col_name]
ins = instructions["ophthafterdxdate"][col_name]

In [21]:
result = PatpopScrub(col, ins).clean()
check = display_check(result)

In [23]:
check

,old_old_VA_Left_Distance_SC,VA_Left_Distance_SC
476,20/60-2,20/60
525,CF at 3',Count fingers
526,CF at 2',Count fingers
527,CF @ 6',Count fingers
528,CF at 7',Count fingers
...,...,...
108220,CF at 3',Count fingers
108223,20/300-turns eye to the right,20/300
108335,CF at 3',Count fingers
108337,20/300 eccentric,20/300


### VA_Right_Distance_CC

In [24]:
col_name = "VA_Right_Distance_CC"

In [26]:
col = df[col_name]
ins = instructions["ophthafterdxdate"][col_name]

In [27]:
result = PatpopScrub(col, ins).clean()
check = display_check(result)

In [28]:
check

,old_old_VA_Right_Distance_CC,VA_Right_Distance_CC
2,CF at 3',Count fingers
3,CF at 1',Count fingers
5,20/800 with near card,20/800
10,CF at 3',Count fingers
11,CF at 3',Count fingers
...,...,...
108239,CF at 2',Count fingers
108242,CF at 4,Count fingers
108367,CF at 1-2',Count fingers
108370,CF at 1-2',Count fingers


### VA_Left_Distance_CC

In [11]:
col_name = "VA_Left_Distance_CC"

In [12]:
col = df[col_name]
ins = instructions["ophthafterdxdate"][col_name]

In [13]:
result = PatpopScrub(col, ins).clean()
check = display_check(result)

In [17]:
ch  = check[~check["VA_Left_Distance_CC"].isna() & check["VA_Left_Distance_CC"].str.contains("not found")]

In [20]:
ch["old_VA_Left_Distance_CC"][11974] == ""

True

### VA_Right_Pressure

In [23]:
col_name = "VA_Right_Pressure"

In [24]:
col = df[col_name]
ins = instructions["ophthafterdxdate"][col_name]

In [25]:
result = PatpopScrub(col, ins).clean()
check = display_check(result)

In [31]:

old, new = result[0], result[1][0]
old_new=pd.DataFrame([old, new]).T
old_new.columns=["old", "new"]

In [32]:
old_new[]

,old,new
0,NaN,NaN
1,NaN,NaN
2,18,no_lookup
3,17,no_lookup
4,NaN,NaN
...,...,...
108369,NaN,NaN
108370,17,no_lookup
108371,19,no_lookup
108372,NaN,NaN


In [29]:
check[check["VA_Right_Pressure"]!="no_lookup"]

,old_VA_Right_Pressure,VA_Right_Pressure
2516,---,NaN
2679,soft,NaN
2680,soft,NaN
3934,x,NaN
6277,Soft,NaN
...,...,...
105685,UTO,NaN
107396,x,NaN
108027,X,NaN
108028,X,NaN


***

In [35]:
old_new = pd.DataFrame([result[0], result[1][0]]).T
old_new.columns = ["old", "new"]

In [37]:
old_new

,old,new
0,NaN,NaN
1,NaN,NaN
2,CF at 3',Count fingers
3,CF at 1',Count fingers
4,NaN,NaN
...,...,...
108369,NaN,NaN
108370,CF at 1-2',Count fingers
108371,CF at 3',Count fingers
108372,NaN,NaN


In [36]:
old_new[(old_new["old"] == old_new["new"])]["new"].value_counts()

new
20/30      5874
20/25      5695
20/40      5208
20/20      4654
20/50      3725
20/60      2361
20/70      1388
20/100     1117
20/80       924
20/200      790
20/400      682
20/150      590
20/125      569
20/300      314
20/500      314
20/250      176
20/63        50
20/600       43
20/16        40
20/32        32
20/160       31
20/800       30
20/1000      18
20/15        17
20/320       10
20/700        1
20/350        1
Name: count, dtype: int64

In [9]:
old_new = pd.DataFrame([result[0], result[1][0]]).T
old_new.columns = ["old", "new"]

In [10]:
rules = get_all_rules(ins["args"])

In [14]:
rules.sort_values("Occurences", ascending=False)

,Value,Rule (to be defined by CCF team),Occurences
100,20/30,20/30,2574
71,20/25,20/25,2488
131,20/40,20/40,2448
43,20/20,20/20,1997
177,20/50,20/50,1702
...,...,...,...
237,20/70+2,20/70,1
233,20/70 first letter only the rest is clumped to...,20/70,1
231,20/70 blurry,20/70,1
230,20/70 Push,20/70,1


In [12]:
args= ins["args"]

In [13]:
num_rules = VARuleReader(
            args["sheet_name"],
            args["num_head"],
            args["num_tail"],
            args["num_cols"],
        )

In [30]:
num_rules.rules["Value"]

0     200
1    2030
2    2040
3      72
Name: Value, dtype: object

In [29]:
num_rules.rules.iloc[0]

Value                                              200
Rule (to be defined by CCF team)    Replace with blank
Occurences                                           1
Name: 0, dtype: object

In [44]:
rules

{'200': 'Replace with blank',
 '2030': '20/30',
 '2040': '20/40',
 '72': 'Replace with blank',
 "2'/200": 'Count fingers',
 '2/200': 'Count fingers',
 '20//60': '20/60',
 '20/100': '20/100',
 '20/100 -1': '20/100',
 '20/100 diverted': '20/100',
 '20/100 looking up': '20/100',
 '20/100 periph': '20/100',
 '20/100 peripherally': '20/100',
 '20/100 slow': '20/100',
 '20/100+': '20/100',
 '20/100+1': '20/100',
 '20/100-1': '20/100',
 '20/100-cloudy': '20/100',
 '20/1000': '20/1000',
 '20/100BCL': '20/100',
 '20/120': '20/120',
 '20/125': '20/125',
 '20/125 ecc fixation': '20/125',
 '20/125 eccentric': '20/125',
 '20/125 eccntrically': '20/125',
 '20/125 peripherally': '20/125',
 '20/125 searching': '20/125',
 '20/125 single letter': '20/125',
 '20/125-': '20/125',
 '20/125-1': '20/125',
 '20/1250': 'Replace with blank',
 '20/15': '20/15',
 '20/15 slow': '20/15',
 '20/15-2': '20/15',
 '20/150': '20/150',
 '20/150 CLB': '20/150',
 '20/150 blurred': '20/150',
 '20/150 eccentric': '20/150',
 '

In [40]:
old_new[(old_new["old"] == old_new["new"])]["new"].value_counts()

new
20/30      2574
20/25      2488
20/40      2448
20/20      1997
20/50      1702
20/60      1146
20/70       738
20/100      609
20/200      561
20/80       510
20/150      417
20/400      365
20/125      302
20/300      153
20/500      151
20/250      100
20/15        24
20/600       16
20/800       11
20/1000      10
20/16         4
20/32         2
20/160        2
20/63         2
20/120        1
20/320        1
Name: count, dtype: int64

In [38]:
old_new[~old_new["old"].isna() & (old_new["old"] == old_new["new"])]

,old,new
9,20/800,20/800
15,20/40,20/40
17,20/60,20/60
18,20/70,20/70
29,20/30,20/30
...,...,...
108329,20/60,20/60
108350,20/30,20/30
108351,20/25,20/25
108353,20/40,20/40


In [46]:
old_new[~old_new["new"].isna() & old_new["old"].str.contains("73")]

,old,new


In [47]:
old_new[old_new["old"] ==73]

,old,new


In [26]:
old_new[~old_new["old"].isna() & (old_new["old"] == old_new["new"])]

,old,new
20,20/70,20/70
21,20/70,20/70
120,20/60,20/60
121,20/20,20/20
122,20/25,20/25
...,...,...
108360,20/60,20/60
108361,20/60,20/60
108362,20/50,20/50
108363,20/70,20/70


In [15]:
rules = get_occs(ins["args"])

In [16]:
rules = pd.DataFrame(rules.groupby(["Value", "Rule (to be defined by CCF team)"])["Occurences"].sum()).reset_index()

In [17]:
def mod_expected(df, col_name, rules, expected):
    rules_merge = pd.merge(rules, df[[col_name]], left_on=["Value"], right_on=[col_name], how="left")
    rules_merge["actual_occurences"] = rules_merge.groupby(col_name)[col_name].transform("size")
    mis_match = rules_merge[rules_merge["Occurences"] != rules_merge["actual_occurences"]]
    dups_dropped = mis_match.drop_duplicates(subset=["Value"])
    nan_count = sum(dups_dropped[dups_dropped["Value"].isnull()]["Occurences"])
    occ_diff = dups_dropped[~dups_dropped["Value"].isnull()]
    act_minus_exp = sum(occ_diff["actual_occurences"]) - sum(occ_diff["Occurences"])

    return expected - nan_count + act_minus_exp
    




In [18]:
rules_merge = pd.merge(rules, df[[col_name]], left_on=["Value"], right_on=[col_name], how="left")

In [25]:
rules_merge

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences
0,-,Replace with blank,2,-,2
1,-,Replace with blank,2,-,2
2,--,Replace with blank,10,--,10
3,--,Replace with blank,10,--,10
4,--,Replace with blank,10,--,10
...,...,...,...,...,...
127,x,Replace with blank,9,x,9
128,x,Replace with blank,9,x,9
129,x,Replace with blank,9,x,9
130,x,Replace with blank,9,x,9


In [19]:
rules_merge["actual_occurences"] = rules_merge.groupby(col_name)[col_name].transform("size")

In [41]:
rules_merge

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences
0,-,Replace with blank,2,-,2
1,-,Replace with blank,2,-,2
2,--,Replace with blank,10,--,10
3,--,Replace with blank,10,--,10
4,--,Replace with blank,10,--,10
...,...,...,...,...,...
127,x,Replace with blank,9,x,9
128,x,Replace with blank,9,x,9
129,x,Replace with blank,9,x,9
130,x,Replace with blank,9,x,9


In [42]:
rules_merge[rules_merge["Value"].isna()]

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [38]:
mis_match = rules_merge[rules_merge["Occurences"] != rules_merge["actual_occurences"]]

In [39]:
mis_match

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [27]:
dups_dropped =mis_match.drop_duplicates(subset=["Value"])

In [28]:
dups_dropped

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [29]:
nan_count = sum(dups_dropped[dups_dropped["Value"].isnull()]["Occurences"])

In [30]:
nan_count

0

In [31]:
occ_diff = dups_dropped[~dups_dropped["Value"].isnull() & ~dups_dropped['actual_occurences'].isnull()]

In [33]:
cc_diff

NameError: name 'cc_diff' is not defined

In [32]:
occ_diff

,Value,Rule (to be defined by CCF team),Occurences,VA_Right_Pressure,actual_occurences


In [34]:
sum(occ_diff["actual_occurences"])

0

In [35]:
sum(occ_diff["Occurences"])

0

In [36]:
sum(occ_diff["actual_occurences"]) - sum(occ_diff["Occurences"])

0

***

In [ ]:
occ_diff = 

In [46]:
rules_merge["Value"].value_counts()

Value
--              20
x               13
soft            12
UTO             11
X                9
---              8
uto              6
CL               6
dfr              5
unab             4
defe             4
NT               4
Soft             3
sft              3
Pros             2
Dfr              2
Prosthetic       1
nl               1
Unable           1
pros             1
16?              1
STP              1
20?              1
18-1             1
`18              1
21-2             1
stp              1
22, 24           1
BCL in place     1
BCL              1
DEF              1
-                1
err              1
0-10             1
<5               1
DFR              1
NTT              1
Name: count, dtype: int64

In [59]:
str_rule_count = str_rules.rules["Occurences"]


In [61]:
str_rule_count.name = "Occurences"

In [64]:
str_rule_count

0     20
1     13
2     12
3     11
4      9
5      8
6      6
7      5
8      4
9      4
10     4
11     3
12     3
13     3
14     2
15     2
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
37     1
38     1
39     1
Name: Occurences, dtype: int64

In [65]:
pd.Series(rules_merge["Value"].value_counts().values, name="Occurences")

0     20
1     13
2     12
3     11
4      9
5      8
6      6
7      6
8      5
9      4
10     4
11     4
12     3
13     3
14     2
15     2
16     1
17     1
18     1
19     1
20     1
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     1
29     1
30     1
31     1
32     1
33     1
34     1
35     1
36     1
Name: Occurences, dtype: int64

In [ ]:
from etls.sheets.va_cols import VARuleReader


rules= VARuleReader()

In [23]:
display_check(result)

,old_VA_Left_Pressure,VA_Left_Pressure
2679,soft,NaN
2680,soft,NaN
2689,X,NaN
5004,--,NaN
6277,Soft,NaN
...,...,...
105684,UTO,NaN
105685,UTO,NaN
105861,---,NaN
107253,defe,NaN


In [19]:
133-129

4

In [13]:
orig

0         NaN
1         NaN
2          19
3          15
4         NaN
         ... 
108369    NaN
108370     17
108371     16
108372    NaN
108373    NaN
Name: VA_Left_Pressure, Length: 108374, dtype: object

In [14]:
ammend

[0         NaN
 1         NaN
 2          19
 3          15
 4         NaN
          ... 
 108369    NaN
 108370     17
 108371     16
 108372    NaN
 108373    NaN
 Name: VA_Left_Pressure, Length: 108374, dtype: object]